<a href="https://colab.research.google.com/github/Imz4k/BOT-JST1/blob/main/KisiUASForum15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import time
import numpy as np

from datetime import datetime
from numpy import genfromtxt
import sys

In [2]:
epoch = 5000
batch_size = 16

In [3]:
import time
class TimerError(Exception):
  """A custom exception used to report errors in use of timer class"""
 
class Timer:
    def __init__(self):
        self._start_time = None
 
    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimeError(f"Timer is running. Use .stop() to stop it")
 
        self._start_time = time.perf_counter()
 
    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
 
        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Executed time: {elapsed.time:0.4f} seconds")
      

In [4]:
A = np.array([])
B = np.array([])
 
df = genfromtxt('dataset_delta_robot.csv',delimiter=',')
for i in range(len(df)):
  if(i>0):
 
    A = np.append(A, [ df[i][2], df[i][3] ]),
    B = np.append(B, [ df[i][4], df[i][5], df[i][6] ])
 
x_train = np.reshape(A, (-1, 2))
y_train = np.reshape(B, (-1, 3))
 
print("data lenght: ", len(x_train))

data lenght:  1000


In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(2, input_dim=2, activation= tf.nn.leaky_relu),
  tf.keras.layers.Dropout(0.2),
 
  tf.keras.layers.Dense(8, activation= tf.nn.leaky_relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation= tf.nn.leaky_relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(8, activation= tf.nn.leaky_relu),
  tf.keras.layers.Dropout(0.2),
 
  tf.keras.layers.Dense(3, activation= tf.nn.leaky_relu) #softmax
])

In [6]:
model.compile(
    optimizer = 'adam',
    loss = "mse", #sparse_categorical_crossentropy
    metrics = ['accuracy']
)

In [7]:
# Using Tensorboard
 
%load_ext tensorboard
 
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tb_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [8]:
%%time
train = model.fit(
    x_train,
    y_train,
    epochs = epoch,
    verbose = 0,
    batch_size = batch_size,
    validation_split = 0.3
)

CPU times: user 8min 18s, sys: 38.8 s, total: 8min 57s
Wall time: 7min


In [9]:
print("num of epoch: ",epoch)
print("AVG Training Accuracy", np.average(train.history['accuracy']))
print("AVG Training Loss", np.average(train.history['loss']))
print("Data Size", sys.getsizeof(df))

num of epoch:  5000
AVG Training Accuracy 0.6802471428394318
AVG Training Loss 235.68308223571776
Data Size 104216


In [10]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6006 (pid 530), started 0:55:38 ago. (Use '!kill 530' to kill it.)

<IPython.core.display.Javascript object>

In [11]:
# model.evaluate
(
      x_train[:901],
      y_train[:901]
)

(array([[ 1.00e+01, -3.28e+02],
        [ 1.00e+01, -3.28e+02],
        [ 1.00e+01, -3.28e+02],
        ...,
        [-1.00e-01, -2.55e+02],
        [-1.00e-01, -2.55e+02],
        [-1.00e-01, -2.55e+02]]), array([[ 70.379,  77.206,  53.967],
        [ 70.213,  76.922,  53.965],
        [ 70.05 ,  76.641,  53.963],
        ...,
        [-11.229, -20.398,  -1.933],
        [-11.222, -20.403,  -1.914],
        [-11.216, -20.409,  -1.894]]))

In [12]:
# Testing to Predicting
pred = np.array([
        x_train[901]
])

 # Calculating teh accuracy from its result
abs = np.absolute(
      np.subtract(
      model.predict(pred)[0],
      y_train[901]
        )
      )
loss = np.divide(abs, y_train[901])
acc = np.absolute(
        np.average(1 - loss)
      )

print('Predictive: ', model.predict(pred)[0])
print('Target: ', y_train[901])

print("AVG Testing Accuracy", acc)

Predictive:  [-26.383482 -24.055813 -13.92229 ]
Target:  [-11.209 -20.415  -1.875]
AVG Testing Accuracy 3.652445981234738
